# 🏛️ Exploración de Datos: Rendimiento Estudiantil (Google Sheets 2024)

In [ ]:
# Instalación de librerías necesarias
!pip install --upgrade gspread gspread-dataframe ydata-profiling

# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default
from ydata_profiling import ProfileReport


In [ ]:
# Autenticación
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Abrir archivo de Google Sheets
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/16goZeoHV7c8HoGmZDNBSjhozXrmOok9sr_J4-XhHDhQ/edit?usp=sharing')
worksheet = spreadsheet.sheet1

# Cargar como DataFrame
df = get_as_dataframe(worksheet)

# Mostrar primeras filas
df.head()


In [ ]:
# Dimensiones
print(f"El dataset tiene {df.shape[0]} filas y {df.shape[1]} columnas.")

# Información general
df.info()

# Estadísticas generales
df.describe(include='all')

# Distribución de datos categóricos
for col in df.select_dtypes(include='object').columns:
    print(f"\nDistribución de {col}:")
    print(df[col].value_counts())


In [ ]:
# Reemplazo de valores nulos
print("Valores nulos por columna antes de limpieza:")
print(df.isnull().sum())

# Eliminamos filas con nulos (por simplicidad)
df_clean = df.dropna()

print("Valores nulos por columna después de limpieza:")
print(df_clean.isnull().sum())

# Reemplazo de duplicados
print(f"Número de filas duplicadas: {df_clean.duplicated().sum()}")
df_clean = df_clean.drop_duplicates()
print(f"Después de eliminar duplicados: {df_clean.duplicated().sum()}")

# Codificar sexo (opcional)
df_clean['sexo_encoded'] = df_clean['sexo'].map({'M': 0, 'F': 1})


In [ ]:
# Histograma de Comprensión de Texto y Matemáticas
df_clean[['M500_EM_6P_2024_CT', 'M500_EM_6P_2024_MA']].hist(figsize=(10,8), bins=15)
plt.tight_layout()
plt.show()

# Barras para variables categóricas
for col in ['sexo', 'area', 'gestion2']:
    plt.figure(figsize=(8,4))
    sns.countplot(x=col, data=df_clean)
    plt.title(f'Distribución de {col}')
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# Diagrama de dispersión entre puntajes de Comprensión de Texto y Matemáticas
sns.pairplot(df_clean[['M500_EM_6P_2024_CT', 'M500_EM_6P_2024_MA']])
plt.show()

# Scatterplot dirigido con diferenciación por sexo
plt.figure(figsize=(8,6))
sns.scatterplot(x='M500_EM_6P_2024_CT', y='M500_EM_6P_2024_MA', hue='sexo', data=df_clean)
plt.title('Relación entre Comprensión de Texto y Matemáticas')
plt.xlabel('Puntaje Comprensión de Texto')
plt.ylabel('Puntaje Matemáticas')
plt.show()

# Boxplot de puntaje de Comprensión de Texto por sexo
plt.figure(figsize=(8,6))
sns.boxplot(x='sexo', y='M500_EM_6P_2024_CT', data=df_clean)
plt.title('Comprensión de Texto según Sexo')
plt.show()


In [ ]:
# Crear reporte automático
profile = ProfileReport(df_clean, title="Reporte EDA de Estudiantes 2024", explorative=True)

# Mostrar reporte
profile.to_notebook_iframe()
